In [1]:
# Import Modules
import os, pickle, re, random
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Import Model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
# Read data
data_path = "./data"

with open(data_path + '/self_00.txt', 'r', encoding='utf-8') as f:
    text = f.read()

with open(data_path + '/self_00_q.txt', 'r', encoding='utf-8') as f:
    questions_list = f.read()
    questions_list = questions_list.split('\n')
    questions_list = [q.strip() for q in questions_list if q != '']
print(f"질문의 수 : {len(questions_list)}")

질문의 수 : 3


In [11]:
# 1. 문장 단위 검사
sentences = re.split(r'[.\n]', text)
sentences = [s.strip() for s in sentences] # 문장 단위로 분리
sentences = [s for s in sentences if len(s) > 20]
print(f"데이터의 문장 수 : {len(sentences)}")
print(f"예시 : {sentences[random.randint(0, len(sentences)-1)]}")

embeddings = []
for idx in tqdm(range(len(sentences)), ncols=80):
    embeddings.append(model.encode(sentences[idx]))

for question in tqdm(questions_list, ncols=80):
    question = question.replace('\n', '')
    question_embedding = model.encode(question)
    max_sim = 0; max_idx = -1
    for idx in range(len(embeddings)):
        cur_sim = util.cos_sim(embeddings[idx], question_embedding)
        max_idx = idx if cur_sim > max_sim else max_idx
        max_sim = cur_sim if cur_sim > max_sim else max_sim
    print(f"Q : {question}")
    print(f"A : {sentences[max_idx]} {max_sim}")

데이터의 문장 수 : 4
예시 : 관리 종료 후 7 일 이내 결과보고서 , 출석부 및 계좌 사본 각 1 부 제출


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 122.00it/s]

Q : 관리자는 접속 기간 중 최소 80% 이상 보고서를 작성해야 합니다. 개인적인 이유로 결석이 필요할 경우, 반드시 사전에 본부장의 승인을 받아야 하며, 3일 이상 무단 결석 시 연수비 지원이 중단됩니다.
A : 관리자는 반드시 보고서를 접속 기간의 80% 이상 작성해야 한다 tensor([[0.7659]])
Q : 관리가 종료된 후 7일 이내에 성과 보고서, 출석 기록, 그리고 계좌 사본을 각각 1부씩 제출해야 합니다.
A : 관리 종료 후 7 일 이내 결과보고서 , 출석부 및 계좌 사본 각 1 부 제출 tensor([[0.7222]])
Q : 관리자는 관리자 교육 과정을 필수적으로 이수해야 합니다.
A : 관리자는 반드시 관리자 교육 이수해야 한다 tensor([[0.7900]])


In [81]:
# 2. Window Size 단위 검사
window_size = 100
preprocess_text = text.replace('\n', '')
sentences = []
embeddings = []
for i in tqdm(range(0, len(preprocess_text)-window_size), ncols=80):
    sentences.append(preprocess_text[i:i+window_size])
    embeddings.append(model.encode(preprocess_text[i:i+window_size]))

for question in tqdm(questions_list, ncols=80):
    question = question.replace('\n', '')
    question_embedding = model.encode(question)
    max_sim = 0; max_idx = -1
    for idx in range(len(embeddings)):
        cur_sim = util.cos_sim(embeddings[idx], question_embedding)
        max_idx = idx if cur_sim > max_sim else max_idx
        max_sim = cur_sim if cur_sim > max_sim else max_sim
    print(f"Q : {question}")
    print(f"A : {sentences[max_idx]} {max_sim}")

 25%|███████████▎                                 | 1/4 [00:01<00:03,  1.25s/it]

Q : 지미 카터는 미국의 제 39대 대통령을 역임했어. 그의 취임사를 만들어줘.
A : 램을 확대했다. 또한 환경 운동가의 의견을 따라 조지아주 플린트 강에 댐을 건설하는 계획에 거부권을 행사했다. 카터는 인권 문제를 최우선 과제로 삼았으며, 흑인 주 정부 직원 수를 tensor([[0.8498]])


 50%|██████████████████████▌                      | 2/4 [00:02<00:02,  1.28s/it]

Q : 지미 카터는 1946년부터 1953년까지 대서양과 태평양 함대에 배치되어 버지니아, 하와이, 코네티컷, 뉴욕, 캘리포니아에 거주했어. 그의 인적사항을 이전에 입력한 형식에 맞춰 인쇄해줘.
A : 군 경력.1946년부터 1953년까지 카터 부부는 대서양 과 태평양 함대에 배치되어 버지니아, 하와이, 코네티컷, 뉴욕, 캘리포니아에 거주했다. 1948년 잠수함 임무를 위한 장교 tensor([[0.9357]])


 75%|█████████████████████████████████▊           | 3/4 [00:03<00:01,  1.27s/it]

Q : 지미 카터로 인해 주 의회에 유아 발달 프로그램과 교도소 개혁 프로그램에 대한 예산을 지원해야 해. 조직원들에게 이에 대해 설명하는 이메일을 작성해줘.
A : 게 비난했으나, 미국 정부가 신군부를 설득하는데, 한계가 있었고 결국 묵인하는 듯한 태도를 보이게 됐다.퇴임 이후.퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설 tensor([[0.9130]])


100%|█████████████████████████████████████████████| 4/4 [00:05<00:00,  1.28s/it]

Q : 1946년부터 1953년까지 카터 부부는 대서양 과 태평양 함대에 배치되어 버지니아, 하와이, 코네티컷, 뉴욕, 캘리포니아에 거주했다.
A : 1946년부터 1953년까지 카터 부부는 대서양 과 태평양 함대에 배치되어 버지니아, 하와이, 코네티컷, 뉴욕, 캘리포니아에 거주했다. 1948년 잠수함 임무를 위한 장교 훈련을  tensor([[0.9564]])


In [15]:
# 3. Debug
a = r"관리가 종료된 후 7일 이내에 성과 보고서, 출석 기록, 그리고 계좌 사본을 각각 1부씩 제출해야 합니다."
b = r"관리 종료 후 7 일 이내 결과보고서 , 출석부 및 계좌 사본 각 1 부 제출"

print(util.cos_sim(model.encode(a), model.encode(b)))
print(len(model.encode(b)))

tensor([[0.7222]])
384


: 